Playing games with Spuds

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_cost = Dict(
    ability_imitative => cost_imitative,
    ability_reciprocating => cost_reciprocating,
    ability_romantic => cost_romantic,
    ability_bibliophile => cost_bibliophile,
    ability_melodious => cost_melodious,
    ability_iconoclast => cost_iconoclast,
    ability_intoxicating => cost_intoxicating,
    ability_otaku => cost_otaku,
    ability_sincere => cost_sincere,
    ability_cowardly => cost_cowardly,
)

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

LoadError: UndefVarError: `cost_imitative` not defined

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

In [10]:
function sample_library(ss_prob, as, n_init = 1000000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for a1 in as
        for h in hrange
            for f in frange
                for l in lrange
                    for p in prange
                        for r in rrange
                            for s in srange
                                if rand() <= ss_prob
                                    ff = Spud("", h,f,l,p,r,s,a1)
                                    ff_i += 1
                                    ffs[ff_i] = ff
                                end
                            end
                        end                        
                    end                        
                end
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

sample_library (generic function with 2 methods)

In [11]:
struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

ffp2 (generic function with 1 method)

## Battle code

In [12]:
mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

In [13]:
n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function modify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, mod::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + mod[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = modify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = modify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [138]:
function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


ai2_create (generic function with 1 method)

In [15]:
# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_greedy_pick (generic function with 1 method)

In [16]:
# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_conservative_pick (generic function with 1 method)

In [17]:
# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

ai_myopic_pick (generic function with 1 method)

In [18]:
creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

play_game (generic function with 2 methods)

In [19]:
# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

In [20]:
@time lib = sample_library(1.01, [999]);

  0.287566 seconds (62 allocations: 249.058 MiB, 9.98% gc time)


In [103]:
function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated (generic function with 1 method)

## Generating p.win data by calculation

### Tensor-cdf computations

In [139]:
function computeTCDF6(tpdf::Array{Int64})::Array{Int64}
    tcdf = 0 .* tpdf
    for i1 in 1:size(tpdf)[1]
        for i2 in 1:size(tpdf)[2]
            for i3 in 1:size(tpdf)[3]
                for i4 in 1:size(tpdf)[4]
                    for i5 in 1:size(tpdf)[5]
                        for i6 in 1:size(tpdf)[6]
                            tcdf[i1,i2,i3,i4,i5,i6] = sum(tpdf[1:i1,1:i2,1:i3,1:i4,1:i5,1:i6])
                        end
                    end
                end
            end
        end    
    end
    return tcdf
end

function integrate_cdf(tcdf::Array{Int64}, mins::Array{Int64}, maxes::Array{Int64}, fixed::Array{Int64} = Array{Int64}(undef, 0))::Int64
    if length(mins) == 0
        if minimum(fixed) == 0
            return 0
        else
            return tcdf[CartesianIndex(tuple(fixed...))]
        end
    end
    new_min = mins[1]
    new_max = maxes[1]
    new_fixed1 = append!(copy(fixed), [new_min])
    new_fixed2 = append!(copy(fixed), [new_max])
    ub = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed2)
    lb = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed1)
    return ub - lb
end

integrate_cdf (generic function with 2 methods)

In [133]:
fixed = []
mins = [1,2,3]
maxes = [2,4,6];
new_min = mins[1]
new_max = maxes[1]
new_fixed1 = append!(copy(fixed), [new_min])
println(new_fixed1)
new_fixed2 = append!(copy(fixed), [new_max])

Any[1]


1-element Vector{Any}:
 2

In [118]:
tpdf = ones(Int64, (9,9,9,9,9,9));

In [122]:
tcdf = computeTCDF6(tpdf);

In [ ]:
# cdf([a,b]x[c,d]) = cdf(b, [c,d]) - cdf[a, [c,d]]
# cdf(b, [c,d]) = cdf(b,d) - cdf(b,c)


In [131]:
v = [1,2,3,4,5,6]
tcdf[CartesianIndex(tuple(v...))]

720

In [144]:
integrate_cdf(tcdf, [0,1,2,1,2,0], [3,3,3,3,3,3])

36

### Generate Julia code for computing p-win from a tensor-cdf

In [277]:
# spuds to min-max matrix
function spuds_to_mmm0(as::Array{Spud})::Array{Int64}
    df = spuds_to_df(as)
    mat = zeros(Int64, (length(as), 12))
    mat[:, 1] = df.h
    mat[:, 3] = df.f
    mat[:, 5] = df.l
    mat[:, 7] = df.p
    mat[:, 9] = df.r
    mat[:,11] = df.s;
    mat[:, 2] = df.h
    mat[:, 4] = df.f
    mat[:, 6] = df.l
    mat[:, 8] = df.p
    mat[:,10] = df.r
    mat[:,12] = df.s;
    return mat
end

function spuds_to_mmm(as::Array{Spud})::Array{Int64}
    mmm = spuds_to_mmm0(as);
    for j in [6,5,4,3,2,1]
        inds_used = zeros(Int64, size(mmm)[1]);
        mmm2 = Array{Int64}(undef, (size(mmm)[1], 12));
        n_mmm2 = 0;
        while minimum(inds_used) == 0
            ind_next = findall(inds_used .== 0)[1]
            v = mmm[ind_next, :];
            eq = ones(Int64, size(mmm)[1]);
            eq = eq .* (mmm[:, (2 * j -1)] .== mmm[:, (2 * j)])
            for k in 1:6
                if k != j
                    eq = eq .* (mmm[:, (2 * k -1)] .== v[(2 * k -1)])
                    eq = eq .* (mmm[:, (2 * k)] .== v[(2 * k)])
                end
            end
            mmsub = mmm[eq .== 1, :]
            minv = minimum(mmsub[:, 2*j])
            maxv = maximum(mmsub[:, 2*j])
            inds_used[eq .== 1] .= 1;
            if size(mmsub)[1] == (maxv - minv + 1)
                vnew = copy(v)
                vnew[2 * j - 1] = minv
                vnew[2 * j] = maxv
                n_mmm2 += 1
                mmm2[n_mmm2, :] = vnew
            else
                nsub = size(mmsub)[1]
                mmm2[(n_mmm2+1):(n_mmm2+nsub), :] = mmsub
                n_mmm2 += nsub
            end
        end
        mmm2 = mmm2[1:n_mmm2, :]
        mmm = mmm2;
    end
    return mmm
end

spuds_to_mmm (generic function with 1 method)

In [145]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:3
frange = 1:3
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax3 = ffs;

In [146]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3 (except for H, F where MXV = 4)
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:4
frange = 1:4
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax4 = ffs;

In [281]:
## case 1. H = F
ff = Spud("",2,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax3];
i_beat = libmax3[evs .== 1]
i_tie = libmax3[evs .== 0]
length(i_beat)
mmm_h_eq_f = spuds_to_mmm(i_beat);

In [282]:
## case 2. H > F
ff = Spud("",3,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = filter_nondominated(libmax4[evs .== 1])
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_gt_f = spuds_to_mmm(i_beat);

In [284]:
## case 3. H < F
ff = Spud("",2,3,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = filter_nondominated(libmax4[evs .== 1])
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_lt_f = spuds_to_mmm(i_beat);

## Generating p.win data by brute force

In [81]:
length(lib)

531441

Round 1 of filtering

In [88]:
ref = [rand(lib) for i in 1:100]
@time scores = [eval_battle_list(ff, ref) for ff in lib];
scores = scores./length(ref);
scores_rep0 = scores;

 40.534909 seconds (769.61 M allocations: 66.847 GiB, 23.68% gc time, 0.06% compilation time)


In [89]:
ref = [rand(lib) for i in 1:100]
@time scores = [eval_battle_list(ff, ref) for ff in lib];
scores = scores./length(ref);
scores_rep1 = scores;

 41.490495 seconds (770.64 M allocations: 66.924 GiB, 22.88% gc time, 0.07% compilation time)


In [90]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.44

In [91]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [92]:
# filter library step 1
lib2 = lib[scores .>= - 0.5 .&& scores .<=  0.5]
length(lib2)

327125

Round 2 of filtering

In [87]:
ref = [rand(lib) for i in 1:500]
@time scores = [eval_battle_list(ff, ref) for ff in lib2];
scores = scores./length(ref);
scores_rep0 = scores;

 76.811622 seconds (1.44 G allocations: 124.878 GiB, 19.24% gc time, 0.03% compilation time)


In [88]:
ref = [rand(lib) for i in 1:500]
@time scores = [eval_battle_list(ff, ref) for ff in lib2];
scores = scores./length(ref);
scores_rep1 = scores;

 80.277977 seconds (1.44 G allocations: 125.079 GiB, 18.60% gc time, 0.03% compilation time)


In [89]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.21999999999999997

In [90]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [91]:
lib3 = lib2[scores .>= - 0.2 .&& scores .<= 0.2]
length(lib3)

138759

Round 3

In [92]:
ref = [rand(lib) for i in 1:1000]
@time scores = [eval_battle_list(ff, ref) for ff in lib3];
scores = scores./length(ref);
scores_rep0 = scores;

111.213163 seconds (2.02 G allocations: 175.472 GiB, 18.40% gc time, 0.02% compilation time)


In [93]:
ref = [rand(lib) for i in 1:1000]
@time scores = [eval_battle_list(ff, ref) for ff in lib3];
scores = scores./length(ref);
scores_rep1 = scores;

112.697692 seconds (2.02 G allocations: 175.081 GiB, 17.65% gc time, 0.02% compilation time)


In [94]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.15600000000000003

In [95]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [96]:
lib4 = lib3[scores .>= - 0.1 .&& scores .<= 0.1]
length(lib4)

70351

Round 4

In [98]:
ref = [rand(lib) for i in 1:2000]
@time scores = [eval_battle_list(ff, ref) for ff in lib4];
scores = scores./length(ref);
scores_rep0 = scores;

106.849612 seconds (2.05 G allocations: 177.827 GiB, 17.55% gc time, 0.02% compilation time)


In [99]:
ref = [rand(lib) for i in 1:2000]
@time scores = [eval_battle_list(ff, ref) for ff in lib4];
scores = scores./length(ref);
scores_rep1 = scores;

105.142988 seconds (2.05 G allocations: 177.716 GiB, 17.51% gc time, 0.02% compilation time)


In [100]:
scores = (scores_rep0 .+ scores_rep1) .* 0.5;

In [101]:
maximum(map(abs, scores_rep1 - scores_rep0))

0.135

In [102]:
lib5 = lib4[scores .>= - 0.05 .&& scores .<= 0.05]
length(lib5)

34520

## Evaluate internal fitness

In [93]:
df = spuds_to_df(lib2);

In [94]:
libsub = [rand(lib) for i in 1:2000];
@time evs = [eval_battle_list(ff, libsub) for ff in lib2];
w = evs./length(libsub);
df[:, :vsAll] = w

488.959264 seconds (9.53 G allocations: 826.715 GiB, 20.27% gc time, 0.00% compilation time)


327125-element Vector{Float64}:
 -0.531
 -0.523
 -0.516
 -0.508
 -0.502
 -0.514
 -0.503
 -0.493
 -0.512
 -0.508
 -0.5
 -0.495
 -0.488
  ⋮
  0.416
  0.4305
  0.444
  0.449
  0.409
  0.42
  0.434
  0.415
  0.421
  0.423
  0.426
  0.429

In [95]:
libsub = [rand(lib5) for i in 1:2000];
@time evs = [eval_battle_list(ff, libsub) for ff in lib2];
w = evs./length(libsub);
df[:, :vsSel] = w;

490.564353 seconds (9.59 G allocations: 831.670 GiB, 19.31% gc time, 0.01% compilation time)


In [96]:
CSV.write("temp_desL_evs_vs_zeroset.csv", df)

"temp_desL_evs_vs_zeroset.csv"

In [103]:
ff = Spud("", 3, 7, 1, 1, 9, 9, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.5902085747392816

In [104]:
ff = Spud("", 9, 7, 2, 2, 2, 4, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.583082271147161

In [109]:
ff = Spud("", 9, 7, 1, 1, 1, 1, 999)
eval_battle_list(ff, lib5)/length(lib5)

0.3447276940903824

## Use regression coeffs to predict Zero-Zero set

In [49]:
df = CSV.read("temp_desL_init_zerosubset.csv", DataFrame);
df[!, :name] .= "noname"
lib5 = df_to_spuds(df);

In [50]:
ff = rand(lib5)
ev = eval_battle_list(ff, lib)/length(lib)

-0.017145835567824085

In [51]:
using MAT

In [52]:
vars = matread("temp_desL_regress.mat")

Dict{String, Any} with 3 entries:
  "b0" => -0.528944
  "b2" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.00190…
  "b1" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0141574 0.0152…

In [53]:
b0 = vars["b0"];
b1 = vars["b1"];
b2 = vars["b2"];

function predict_ev(a::Spud, b0::Float64, b1::Array{Float64}, b2::Array{Float64})::Float64
    s = b0
    x = [a.h, a.f, a.l, a.p, a.r, a.s]
    for i in 1:6
        s += b1[i, x[i] + 1]
    end
    for i in 1:5
        for j in (i+1):6
            s += b2[i,j,x[i]+1,x[j]+1]
        end
    end
    return s
end

predict_ev (generic function with 2 methods)

In [86]:
ff = rand(lib)
println(ff)
predict_ev(ff,b0,b1,b2)

Spud("", 4, 1, 5, 6, 8, 3, 999)


-0.3595682845475866

In [87]:
@time ev = eval_battle_list(ff, lib5)/length(lib5)

  0.041915 seconds (503.35 k allocations: 44.723 MiB, 38.19% gc time)


-0.8001738122827347